## 작업형 제 2 유향

### ✏️ 건물 데이터를 기반으로 건물의 난방 부하(에너지 효율성)를 예측하시오.

- 제공된 데이터 목록 : train.csv, test.csv

- 예측할 컬럼 : Heat_Load(Very Low, Low, Medium, High, Very High)

- 학습용 데이터(energy_train.csv)를 이용하여 난방 부하 결과를 예측하는 모델을 만든 후 이를 평가용 데이터(energy_test.csv)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.

    - 제출 파일은 다음 1개의 컬럼만 포함해야 한다

        - pred : 예측 결과

        - 제출 파일명 : result.csv

    - 제출한 모델의 성능은 Macro F1-Score 평가지표에 따라 채점

In [ ]:
# 1. 문제 정의
## 에너지 효율성을 평가하기 위한 난방 부하 예측

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv('./data/energy_train.csv')
test = pd.read_csv('./data/energy_test.csv')

# 3. 탐색적 데이터 분석(EDA)
## 데이터 크기
print(train.shape, test.shape, '\n')

## 데이터 샘플
print(train.head(), '\n')
print(test.head(), '\n')

## 데이터 자료형(타입)
print(train.info())

## 기초통계량
print(train.describe(), '\n')

## object 형의 unique 개수
print(train.describe(include='object'), '\n')
print(test.describe(include='object'), '\n')

## 결측치 여부
print(train.isnull().sum().sum(), '\n')
print(test.isnull().sum().sum(), '\n')

## Heat_Load 컬럼의 종류에 따른 개수
print(train['Heat_Load'].value_counts())

(537, 10) (231, 9) 

   Compac  Surf_Area  Wall_Area    Roof Height Orient  Glaze_Area  \
0    0.74      686.0      245.0   220.5  Short  South        0.25   
1    0.98      514.5      294.0   Small   Tall  South        0.40   
2    0.90      563.5      318.5  Medium   Tall   West        0.25   
3    0.74      686.0      245.0   220.5  Short  South        0.25   
4    0.98      514.5      294.0   Small   Tall   East        0.25   

   Glaze_Distr  Cool_Load Heat_Load  
0            3      14.72  Very Low  
1            2      33.94      High  
2            1      37.58      High  
3            2      15.18  Very Low  
4            2      29.69      High   

   Compac  Surf_Area  Wall_Area    Roof Height Orient  Glaze_Area  \
0    0.64      784.0      343.0   220.5  Short  South        0.40   
1    0.82      612.5      318.5   Large   Tall  North        0.40   
2    0.76      661.5      416.5  Medium   Tall  South        0.10   
3    0.74      686.0      245.0   220.5  Short   East     

In [ ]:
# 4. 데이터 전처리
## 인코딩 전 예측할 값이 object 라면 다른 변수에 옮겨두기
## train 에는 Heat_Load 가 있고, test 에는 없기 때문에, 원-핫 인코딩시 컬럼수가 달라지게 됨
target = train.pop('Heat_Load')
print(train.shape, test.shape)

train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)

(537, 9) (231, 9)
(537, 16) (231, 16)


In [ ]:
# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(429, 16) (108, 16) (429,) (108,)


In [13]:
# 6. 머신러닝 학습 및 평가
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
!pip install xgboost
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import f1_score

dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train)
pred = dt.predict(X_val)
print('의사결정 나무 :', f1_score(y_val, pred, average='macro'))

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred = rf.predict(X_val)
print('랜덤 포레스트 :', f1_score(y_val, pred, average='macro'))

# target 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_adjusted = le.fit_transform(y_train)

# xgboost 모델 학습 및 예측 : xgboost 모델은 예측 클래스가 0부터 시작해야 함
xg = xgb.XGBClassifier(random_state=0)
xg.fit(X_train, y_train_adjusted)
pred = xg.predict(X_val)

# 예측 값을 원래 문자로 변경
pred = le.inverse_transform(pred)

print("xgboost :", f1_score(y_val, pred, average='macro'))

lg = lgb.LGBMClassifier(random_state=0, verbose=-1)
lg.fit(X_train, y_train)
pred = lg.predict(X_val)
print('LightGBM :', f1_score(y_val, pred, average='macro'))

의사결정 나무 : 0.9167995817564094
랜덤 포레스트 : 0.9277616846430405
xgboost : 0.9374839068652628
LightGBM : 0.9319703995747778


In [ ]:
# 7. 예측 및 결과 파일 생성
## xgboost 모델 선택해 제출하려면 인코딩을 한 번 더 해야하는 번거로움이 있으므로 사용 어려움
## LightGBM 도 0.9 점대로 높은 성능을 보이고 있어 이것으로 제출해도 OK
pred = lg.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result.csv', index=False)

print(pd.read_csv('result.csv').head())

   pred
0   Low
1  High
2  High
3   Low
4   Low


In [18]:
# 8. 크로스 밸리데이션

# K-fold 크로스 밸리데이션(Cross-Validation)
## train 데이터를 K개로 동일한 크기로 나눈 후 나눈 데이터를 K개의 세트라 부를 때, 각각의 세트를 모두 검증용으로 한 번씩 사용하면서
## 나머지 K-1개 세트를 훈련 데이터로 사용
## 모든 데이터가 훈련 데이터와 검증 데이터로 사용되도록 함
## ex. K가 5라면 이 과정은 다섯 번 거쳐 5개의 평가 결과를 얻게 되고 최종적으로 5개의 평가 결과의 평균을 내어 모델의 성능 평가
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

train = pd.read_csv('./data/energy_train.csv')
test= pd.read_csv('./data/energy_test.csv')

# 데이터 전처리
target = train.pop('Heat_Load')

# 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 모델 준비
rf = RandomForestClassifier(random_state=0)

# 크로스 밸리데이션 수행(K-fold)
scores = cross_val_score(rf, train, target, cv=5, scoring='f1_macro')
print(scores)

# 평균 F1 스코어 출력
print(scores.mean())

# 모델 학습 및 예측
## 크로스 밸리데이션은 모델의 성능을 평가하는 단계에 불과하므로 모델 학습과 예측은 별도 진행
## 크로스밸리데이션에서 평가된 랜덤포레스트 모델을 사용해 전체 데이터에 대해 다시 학습 진행 후 예측 결과를 csv 파일로 저장
rf.fit(train, target)
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result.csv', index=False)

print(pd.read_csv('result.csv').head())

[0.91726901 0.90856297 0.92346084 0.95283204 0.95716959]
0.9318588885917534
   pred
0   Low
1  High
2  High
3   Low
4   Low
